# Bringing out brain regions

## Description

This tool allows us to **hightlight one or more regions of interest in the mouse brain** and visualize them in coronal, sagittal or transversal section.    
The mouse brain reference volumes are a pair of volumetric files from the Allen Institute for Brain Science (**AIBS**): the Nissl and the annotation volumes. 
More specifically:   
* the **Nissl volume** shows every cell in the brain and it is used to obtain the cell distribution
* the **annotation volume** provides a brain region ID for each voxel of the Nissl volume and any volumetric dataset of the AIBS. 

The mouse brain regions are organized in a hierarchical tree and this **hierarchy** is stored in a **json file**, common to each reference atlas version (it is also available on the AIBS website). In this hierarchical tree every region is divided in subregions wich, in turn, are divided in other subregions and so on, with the leaves of the tree representing the finest subdivisions captured by the AIBS.  


Here are some preliminary steps: this tool needs some modules to work properly, so we need to import them first

In [ ]:
import numpy as np
from voxcell import RegionMap, VoxelData
from os.path import join
from matplotlib.pylab import plt
import ipywidgets


Set below (**DATA_FOLDER**) the path to the folder containing the data, including the annotations volumes, the Nissl volumes and the json hierarchy file.

Set also the folder in which the figures produced will be stored (**OUTPUT_FOLDER**).


In [ ]:
DATA_FOLDER = 'path/to/data/'
OUTPUT_FOLDER = "path/to/output"

The data required to run the notebook will be provided before the hands-on session. 
It can also be download if directly on the AIBS website at the following links:
- http://api.brain-map.org/api/v2/structure_graph_download/1.json
- http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/mouse_2011/annotation_25.nrrd
- http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/ara_nissl/ara_nissl_25.nrrd

These three files need to be put in your **DATA_FOLDER**.

Now, we load the following files: 
* the json file containing the hierarchy of the brain regions
* the annotation volume file 
* the nissl volume file 


In [ ]:
region_map = RegionMap.load_json(join(DATA_FOLDER, "1.json"))
annotation = VoxelData.load_nrrd(join(DATA_FOLDER, "annotation_25.nrrd")).raw
nissl = VoxelData.load_nrrd(join(DATA_FOLDER, "ara_nissl_25.nrrd")).raw

In the following section, wecreate a function to convert a Hexadecimal color into its RGB value counterpart; you will need this later.

In [ ]:
def hex_to_rgb(value):
    """
    Converts a Hexadecimal color into its RGB value counterpart.

    Arguments:
        value: string hexadecimal color to convert.
    Returns:
        List of the Red, Green, and Blue components of the color
    """

    value = value.lstrip('#')
    lv = len(value)
    return np.array([int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3)])


## Learn how to find regions in the hierarchy

To do the plot, we need to be able to find informations about the regions that we want to highlight. 
In this section, you will learn how to navigate easily into the hierarchical organisation of the mouse brain with the *voxcell* module.  

#### Find IDs in the annotation volume
First of all, let's display the documentation about the function *region_map.find*

In [ ]:
region_map.find?

This function allows ous to search regions matching your request. It will return a set of IDs of regions matching the request. 

We can do a request on the following attributes: 
* **name**: the name of the region
* **acronym**: the acronym of the region  
* **atlas ID**: additional ID only used by the AIBS 
* **color_hex_triplet**: the hexadecimal color code associated to the region 
* **graph_order**: the position of the region in the file
* **st_level**: the hierarchy level associated to the region (the root region in 0 and the finest parcellations are 11)
* **hemisphere_id**: 3 if the region is on both hemispheres  
* **parent_structure_id**: ID of the parent region in the hierarchy 

These attributes correspond to the fields of each region within the json hierarchy file.
 For exemple, for the isocortex region:
```json
{
               "id": 315,
               "atlas_id": 746,
               "ontology_id": 1,
               "acronym": "Isocortex",
               "name": "Isocortex",
               "color_hex_triplet": "70FF71",
               "graph_order": 5,
               "st_level": 5,
               "hemisphere_id": 3,
               "parent_structure_id": 695,
               "children": [...]  # some regions have children or subregions and they will be listed here            
}
```
Let's test the function, with a simple example.

There are two required parameters for the function: the attribute (attr) and the value (value).
Choose the type of attribute in *\"attribute\"*.  After that, put the corresponding value in *\"region_info\"* (it should correspond to the value given in the json file). For example, if the attribute is "name", the parameter "region_info" should be the name of the region of interest.
When you run, the algorithm will return the corresponding ID.  

In [ ]:
attribute = "name"
region_info = "Isocortex"
region_map.find(region_info, attr=attribute)

If you are searching for text values (e.g.: for the attributes "name", "acronym", etc.), the function requires you to provide the text taking into account the case (i.e: the presence or absence of capital letters).
Fortunately, there is a parameter in the find function to ignore the case: *\"ignore_case\"*. 

In [ ]:
attribute = "name"
region_info = "ISoCoRTex"
region_map.find(region_info, attr=attribute, ignore_case=True)

You can also search using a regular expression (https://docs.python.org/3/howto/regex.html) for text values. For exemple, here we are searching for every region containg "cortex" in their name. So it should return not only isocortex but all the regions matching the regular expression. 

In [ ]:
attribute = "name"
region_info = "@.*cortex.*$"
region_map.find(region_info, attr=attribute, ignore_case=True)

Finally, there is the option to return the descendence of the regions you are searching. You can add the parameter *\"with_descendants\"* in the find function. Descendant regions correspond to subregions contained into the region of interest. 

In [ ]:
attribute = "name"
region_info = "Isocortex"
region_map.find(region_info, attr=attribute, ignore_case=True, with_descendants=True)

#### Find values according to IDs
Once we have found the ID of the region of interest, we can also obtain information associated with it. 

First of all, let's display the documentation about the function *region_map.get*

In [ ]:
region_map.get?

Now you can try it. Write down an ID (in "region_ID") and the type of attribute that you want to display (in "attribute")

In [ ]:
region_ID = 315
attribute = "name" # "acronym", etc.
region_map.get(region_ID, attribute)

#### Exercises

Here are two exercises for you to see if you have understood everything: 
1) find all regions that belong to the hippocampus (you may have to open the json hierarchy file to find its correct name)
2) use the function region_map.get to find the name of the parent region of the region cerebellum.

In [ ]:
# TYPE YOUR CODE HERE 

#### Answers to exercises:
Double click on this cell to display the answers
<!---
1)   {19,375,382,391,399,407,415,423,431,438,446,454,463,471,479,486,495,504,632,726,734,742,751,758,766,775,782,790,799,807,815,823,982,1080,10702,10703,10704} 

2) "Basic cell groups and regions"
--->



Congratulations! Now you know your way around the hierarchy.

## Parameters for the plot

Now that you know how the hierarchy works, we are going to plot the annotation volume on top of the Nissl volume, selecting a list of region to highlight. 
This is the interesting part and where you can play: choose below one or more regions that you want to highlight!  

*Be careful*: the name of the region has to be written in the same way as it is in the json file, but it not necessarily has to be the full name 

In [ ]:
regions_to_highlight = ["raphe nuclei"]

### Filtering the regions of interest

In the following part, the algorithm picks up the IDs of the selected regions (in *\"region_to_highlight\"*) and their descendants. 

In [ ]:
ids_region = []
for region_name in regions_to_highlight:
    ids_roi=list(region_map.find("@.*{}.*$".format(region_name), 
                                           attr="name", with_descendants=True, 
                                           ignore_case=True))

    # if the name does not correspond to an existing region in the json file, an error message will be displayed 
    if len(ids_roi)==0:
        print("WARNING: No region found for: {}".format(region_name))  
        continue
    ids_region.append(ids_roi)

if len(ids_region)==0:
    print("WARNING: No region selected, please select at least one valid region")

Now, it assigns a color for each voxel in the Annotation volume corresponding to the IDs in *region_to_highlight*

In [ ]:
colored_annotations = np.full(annotation.shape + (4,), 1.)
colored_annotations[:,:,:,3] = 0.
first_pos = np.zeros(3) # starting slice positions for each axis (coronal, axial, sagittal) for the figure ("default slice")
max_counts = np.zeros(3) # count maximum number of voxel in slice corresponding to the regions of interest for each axis 

for id_reg in ids_region:
    ann_filt = np.isin(annotation, id_reg) 
    color = hex_to_rgb(region_map.get(id_reg[0], attr="color_hex_triplet")) / 255. # get the color of one of the regions of the list (the regions of interest and its subregions). 
    colored_annotations[ann_filt, :3] = color # set rgb color of the region 
    colored_annotations[ann_filt, 3] = 0.5 # set transparence 

    # the following lines ensure that the default slice (pictured in the next section) contains the maximum number of voxel of the selected regions:
    for i, dim in enumerate(np.where(ann_filt)):
        u, c = np.unique(dim, return_counts=True)
        argmax_c = np.argmax(c) 
        max_c = c[argmax_c]
        if max_c > max_counts[i]:
            first_pos[i] = u[argmax_c]
            max_counts[i] = max_c

## Visualize brain slices

Now we can display brain slices with the selected region highlighted!
After running the following lines, you will see the the default *coronal slice*, containing the maximum number of voxel corrisponding to the selected regions, with these regions colored. You can choose the desired slice moving the cursor back and forth, holding down the left mouse button until you reach it. In the "direction" toolbar above the slice, you can choose to display:
* **sagittal slices**
* **coronal slices**
* **axial slices**

In [ ]:
cmap = plt.cm.gray  # color map for Nissl
axes_ = ["coronal", "axial", "sagittal"]
@ipywidgets.interact
def f_final_outer(direction=ipywidgets.Dropdown(options=axes_)):
    direction = axes_.index(direction)
    @ipywidgets.interact
    def f_final_inner(slice_position=ipywidgets.IntSlider(value=first_pos[direction],
                                             min=0, max=annotation.shape[direction] - 1, 
                                             continuous_update=False)):
        fig, ax = plt.subplots(figsize=(15, 9.1))
        plt.suptitle('Section number: {}'.format(slice_position))
        if direction ==2:
            ax.imshow(np.take(1-nissl, slice_position, axis=direction).T, cmap=cmap, interpolation="nearest")
            ax.imshow(np.moveaxis(np.take(colored_annotations, slice_position, axis=direction), 0, 1), interpolation="nearest")
        else:
            ax.imshow(np.take(1-nissl, slice_position, axis=direction), cmap=cmap, interpolation="nearest")
            ax.imshow(np.take(colored_annotations, slice_position, axis=direction), interpolation="nearest")
        ax.axis('off')

You can save a figure in the output folder: choose the slice position (parameter "slice_position") and the direction of the axis and it will be saved in your folder. For the direction, write: 
* 0 for the coronal axis
* 1 for the axial axis 
* 2 for the sagittal axis

In [ ]:
axes_ = ["coronal", "axial", "sagittal"]
slice_position = 300
direction = 0 # 0 for coronal, 1 for axial and 2 for sagittal

fig, ax = plt.subplots(figsize=(15, 9.1))
if direction ==2:
    ax.imshow(np.take(1-nissl, slice_position, axis=direction).T, cmap=cmap, interpolation="nearest")
    ax.imshow(np.moveaxis(np.take(colored_annotations, slice_position, axis=direction), 0, 1), interpolation="nearest")
else:
    ax.imshow(np.take(1-nissl, slice_position, axis=direction), cmap=cmap, interpolation="nearest")
    ax.imshow(np.take(colored_annotations, slice_position, axis=direction), interpolation="nearest")
ax.axis('off')
plt.savefig(join(OUTPUT_FOLDER, "{}_slice_{}_nissl_ann.png".format(axes_[direction], slice_position)), dpi=200)

You can see brain regions in 3D on the Blue Brain Cell Atlas website at:  https://bbp.epfl.ch/nexus/cell-atlas/ 